In [ ]:
# Tracing (Phoenix)
# phoenix serve

import os
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.pydantic_ai import OpenInferenceSpanProcessor
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.trace import set_tracer_provider

# os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://localhost:6006"
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:6006"

# Set up the tracer provider
tracer_provider = TracerProvider()

# Add the OpenInference span processor
endpoint = f"{os.environ['OTEL_EXPORTER_OTLP_ENDPOINT']}/v1/traces"

exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(OpenInferenceSpanProcessor())
tracer_provider.add_span_processor(SimpleSpanProcessor(exporter))

set_tracer_provider(tracer_provider)

from pydantic_ai import Agent
Agent.instrument_all()

In [ ]:
# Tracing (MLFlow)
# mlflow server --host 0.0.0.0 --port 5000

import mlflow

mlflow.pydantic_ai.autolog()
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Pydantic AI")
mlflow.tracing.disable_notebook_display()

In [ ]:
import os
import random
from dotenv import load_dotenv
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel, GoogleProvider
from pydantic_ai.settings import ModelSettings
from pydantic_ai.usage import UsageLimits

FREE_MODELS = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash",
    "gemini-2.0-flash-lite",
    "gemma-3-27b-it",
    "gemma-3n-e4b-it",
    "gemini-embedding-001"
]

load_dotenv()
gemini_pro = GoogleModel(
    "gemini-2.5-pro",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)
gemini_flash = GoogleModel(
    "gemini-2.5-flash",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)
gemini_flash_lite = GoogleModel(
    "gemini-2.5-flash-lite-preview-06-17",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)
gemma = GoogleModel(
    "gemma-3-27b-it",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)

## Basic Functionality

In [ ]:
agent = Agent(gemini_flash_lite)
result = await agent.run("What is the capital of Italy?")
print(result.output)

In [ ]:
INSTRUCTIONS = """\
Answer the user's question using one of the tools provided."""

async def add_numbers(x: int, y: int) -> str:
    """
    Add two numbers together.

    Args:
        x: First number
        y: Second number
    """
    return str(x + y)

async def get_weather(city: str, state: str) -> str:
    """
    Get weather forecast for a location.

    Args:
        city: Name of the city
        state: State abbreviation (e.g., 'CA' for California)
    """
    return f"The weather in {city}, {state} is 75 degrees and sunny."

agent = Agent(
    gemini_flash_lite,
    instructions=INSTRUCTIONS,
    tools=[add_numbers, get_weather]
)
result = await agent.run(
    "What's the weather like in Columbus, Ohio?",
    usage_limits=UsageLimits(request_limit=3)
)
print(result.output)

In [ ]:
result2 = await agent.run(
    "What about Dallas, Texas?",
    message_history=result.new_messages(),
    usage_limits=UsageLimits(request_limit=3)
)
print(result.output)